**MICE** - Multivariate Imputation by Chained Equations

This technique is used when the data is MAR - Missing At Random

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [3]:
df = np.round(pd.read_csv('50_Startups.csv',usecols=['R&D Spend','Administration','Marketing Spend','Profit'])/10000)

In [4]:
df  = df.head()

In [5]:
x = df.drop(columns=['Profit'])
y = df['Profit']

In [6]:
x.iloc[1,0]=np.nan
x.iloc[3,1]=np.nan
x.iloc[-1,-1]=np.nan

In [7]:
x.head()

,R&D Spend,Administration,Marketing Spend
0,17.0,14.0,47.0
1,NaN,15.0,44.0
2,15.0,10.0,41.0
3,14.0,NaN,38.0
4,14.0,9.0,NaN


In [8]:
x.isnull().sum()

,0
R&D Spend,1
Administration,1
Marketing Spend,1


In [9]:
# Imputing all missing values with the column mean value

df_mean = pd.DataFrame()
df_mean['R&D Spend'] = x['R&D Spend'].fillna(x['R&D Spend'].mean())
df_mean['Administration'] = x['Administration'].fillna(x['Administration'].mean())
df_mean['Marketing Spend'] = x['Marketing Spend'].fillna(x['Marketing Spend'].mean())

In [10]:
df_mean.isnull().sum()

,0
R&D Spend,0
Administration,0
Marketing Spend,0


In [11]:
df_mean.head()

,R&D Spend,Administration,Marketing Spend
0,17.0,14.0,47.0
1,15.0,15.0,44.0
2,15.0,10.0,41.0
3,14.0,12.0,38.0
4,14.0,9.0,42.5


In [12]:
# Remove the column1 imputed values
df1 = df_mean.copy()
df1.iloc[1,0]=np.nan
df1.isnull().sum()

,0
R&D Spend,1
Administration,0
Marketing Spend,0


In [13]:
df1

,R&D Spend,Administration,Marketing Spend
0,17.0,14.0,47.0
1,NaN,15.0,44.0
2,15.0,10.0,41.0
3,14.0,12.0,38.0
4,14.0,9.0,42.5


In [14]:
# use first 3 rows(0,2,3) to build a amodel and use the last row for prediction
x  = df1.iloc[[0,2,3],1:3]
x

,Administration,Marketing Spend
0,14.0,47.0
2,10.0,41.0
3,12.0,38.0


In [15]:
y = df.iloc[[0,2,3],0]
y

,R&D Spend
0,17.0
2,15.0
3,14.0


In [17]:
lr = LinearRegression()
lr.fit(x,y)
lr.predict(df1.iloc[1,1:].values.reshape(1,2))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([16.])

In [18]:
df1.iloc[1,0] = 16

In [19]:
df1

,R&D Spend,Administration,Marketing Spend
0,17.0,14.0,47.0
1,16.0,15.0,44.0
2,15.0,10.0,41.0
3,14.0,12.0,38.0
4,14.0,9.0,42.5


In [21]:
df1.iloc[3,1] = np.nan
df1

,R&D Spend,Administration,Marketing Spend
0,17.0,14.0,47.0
1,16.0,15.0,44.0
2,15.0,10.0,41.0
3,14.0,NaN,38.0
4,14.0,9.0,42.5


In [23]:
x  = df1.iloc[[0,1,2,4],[0,2]]
x

,R&D Spend,Marketing Spend
0,17.0,47.0
1,16.0,44.0
2,15.0,41.0
4,14.0,42.5


In [24]:
y = df.iloc[[0,1,2,4],1]
y

,Administration
0,14.0
1,15.0
2,10.0
4,9.0


In [25]:
lr = LinearRegression()
lr.fit(x,y)
lr.predict(df1.iloc[3,[0,2]].values.reshape(1,2))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([9.])

In [26]:
df1.iloc[3,1] = 9

In [27]:
df1

,R&D Spend,Administration,Marketing Spend
0,17.0,14.0,47.0
1,16.0,15.0,44.0
2,15.0,10.0,41.0
3,14.0,9.0,38.0
4,14.0,9.0,42.5


In [28]:
x  = df1.iloc[0:4,0:2]
#x
y = df.iloc[0:4,-1]
#y
lr = LinearRegression()
lr.fit(x,y)
lr.predict(df1.iloc[4,0:2].values.reshape(1,2))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([18.3])

In [29]:
df1.iloc[4,-1] = 18.3

In [30]:
df1

,R&D Spend,Administration,Marketing Spend
0,17.0,14.0,47.0
1,16.0,15.0,44.0
2,15.0,10.0,41.0
3,14.0,9.0,38.0
4,14.0,9.0,18.3


In [31]:
df1 - df_mean

,R&D Spend,Administration,Marketing Spend
0,0.0,0.0,0.0
1,1.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,-3.0,0.0
4,0.0,0.0,-24.2


In [32]:
#these values must be near to 0 there for make a copy of df1 and continue the whole process untill
# we reach near to 0.